In [90]:
import pandas as pd

In [91]:
def format_solution(row, non_unknown_marques):
    marque = non_unknown_marques if row['MARQUES'] == 'UNKNOWN' and row['MODELS'] != 'UNKNOWN' else row['MARQUES']
    model = 'instrument' if row['MODELS'] == 'UNKNOWN' and row['MARQUES'] != 'UNKNOWN' else row['MODELS']
    
    if row['SPECTRUM_TYPE'] == 'UNKNOWN' and row['IONISATION'] != 'UNKNOWN':
        if row['IONISATION'] in ['ESI', 'APPI', 'APCI', 'MALDI', 'FAB', 'FD', 'EI']:
            spectrum_type = 'LC'
        elif row['IONISATION'] in ['CI', 'PTR', 'EI']:
            spectrum_type = 'GC'
        else:
            spectrum_type = row['SPECTRUM_TYPE']
    else:
        spectrum_type = row['SPECTRUM_TYPE']
    
    if row['SPECTRUM_TYPE'] != 'UNKNOWN' and row['IONISATION'] != 'UNKNOWN' and row['INSTRUMENT_TYPE'] == 'UNKNOWN':
        return f"{marque} {model}, {spectrum_type}-{row['IONISATION']}, {row['RESOLUTION']}"
    elif row['SPECTRUM_TYPE'] == 'UNKNOWN' and row['IONISATION'] == 'UNKNOWN' and row['INSTRUMENT_TYPE'] != 'UNKNOWN':
        return f"UNKNOWN, {row['INSTRUMENT_TYPE']}, {row['RESOLUTION']}"
    elif row['SPECTRUM_TYPE'] == 'UNKNOWN' and row['IONISATION'] == 'UNKNOWN' and row['INSTRUMENT_TYPE'] != 'UNKNOWN':
        return f"{marque} {model}, {spectrum_type}-{row['INSTRUMENT_TYPE']}, {row['RESOLUTION']}"
    elif row['SPECTRUM_TYPE'] != 'UNKNOWN' and row['IONISATION'] == 'UNKNOWN' and row['INSTRUMENT_TYPE'] == 'UNKNOWN':
        return f"{marque} {model}, {spectrum_type}, {row['RESOLUTION']}"
    elif row['SPECTRUM_TYPE'] == 'UNKNOWN' and row['IONISATION'] != 'UNKNOWN' and row['INSTRUMENT_TYPE'] == 'UNKNOWN':
        return f"{marque} {model}, {spectrum_type}-{row['IONISATION']}, {row['RESOLUTION']}"
    elif row['SPECTRUM_TYPE'] == 'UNKNOWN' and row['IONISATION'] == 'UNKNOWN' and row['INSTRUMENT_TYPE'] == 'UNKNOWN':
        return f"{marque} {model}, UNKNOWN, {row['RESOLUTION']}"
    elif marque == 'UNKNOWN' and model == 'UNKNOWN':
        return f"UNKNOWN, {spectrum_type}-{row['IONISATION']}-{row['INSTRUMENT_TYPE']}, {row['RESOLUTION']}"
    else:
        return f"{marque} {model}, {spectrum_type}-{row['IONISATION']}-{row['INSTRUMENT_TYPE']}, {row['RESOLUTION']}"

In [92]:
def get_first_non_unknown_marques(df):
    for marque in df['MARQUES']:
        if marque != 'UNKNOWN':
            return marque
    # If there are no non-UNKNOWN marques, just return 'UNKNOWN'
    return 'UNKNOWN'

In [93]:
dfs = pd.read_excel(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\datas\GRAPH_CONSTRUCT.xlsx", sheet_name=None)

In [94]:
# Loop over each DataFrame and add the 'SOLUTION' column
for sheet_name, df in dfs.items():
    df = df.astype(str)
    
    non_unknown_marques = get_first_non_unknown_marques(df)

    df['SOLUTION'] = df.apply(lambda row: format_solution(row, non_unknown_marques), axis=1)

    dfs[sheet_name] = df  # Reassign the modified DataFrame to the original dictionary

In [95]:
# Create an Excel writer
with pd.ExcelWriter(r"C:\Users\Axel\Documents\PYTHON\MSP_V3\FragHub\datas\GRAPH_CONSTRUCT_SOLUTIONS.xlsx", engine='xlsxwriter') as writer:
    # Loop over each DataFrame and write it to a separate sheet
    for sheet_name, df in dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)